# Resistive Wall Wakefield

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pmd_beamphysics.wakefields.resistive_wall import (
    ResistiveWallWakefield,
    epsilon_0,
    apply_sr_wake,
)

from pmd_beamphysics import ParticleGroup
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
radius_ref = 2.5e-3

wake = ResistiveWallWakefield(radius_ref, geometry="round", material="Cu")

wake.plot()
# plt.yscale('symlog')

In [ ]:
print(wake.to_bmad())

# Compare with SLAC-PUB-10707

In [ ]:
!ls ../data

In [ ]:
raw_data = np.loadtxt("../data/SLAC-PUB-10707-digitized-Fig4-AC-Cu.csv", delimiter=",")


# Convert to SI
zref, Wref = (
    raw_data[:, 0] * 1e-6,
    raw_data[:, 1] * 4 / radius_ref**2 / (4 * np.pi * epsilon_0),
)
plt.plot(zref, Wref)

In [ ]:
zlist = np.linspace(0, 300e-6, 200)
Wz = wake.pseudomode(-zlist)

fig, ax = plt.subplots()
ax.plot(zlist * 1e6, Wz * 1e-12, label="ResistiveWallWakefield")

ax.plot(zref * 1e6, Wref * 1e-12, "--", label="Fig4-AC-Cu from SLAC-PUB-10707 2004")
plt.legend()

ax.set_xlabel(r"$-z$ (µm)")
ax.set_ylabel(r"$W_z$ (V/pC/m)")

# Test with Bmad

In [ ]:
LAT = f"""
no_digested
parameter[lattice] = LCLS
parameter[e_tot] = 14e9 
parameter[particle] = electron        
parameter[geometry] = open

beginning[beta_a] = 10
beginning[beta_b] = 10
p1: pipe, L = 1

lat: line = (p1)

P1[sr_wake] = {wake.to_bmad()}

use, lat

"""

In [ ]:
from pytao import Tao

In [ ]:
tao = Tao.from_lattice_contents(LAT, noplot=True)

In [ ]:
beam_init = dict(
    n_particle=1_000,
    a_norm_emit=1e-12,
    b_norm_emit=1e-12,
    sig_z=20e-6,
    sig_pz=1e-9,
    n_bunch=1,
    bunch_charge=1e-9,
    random_engine="quasi",
)

cmds = []
for k, v in beam_init.items():
    cmd = f"set beam_init {k} = {v}"
    cmds.append(cmd)
    print(cmd)

tao.cmds(cmds, suppress_lattice_calc=False)

In [ ]:
%%time
tao.track_beam()

In [ ]:
P0 = ParticleGroup(data=tao.bunch_data("beginning"))
P1 = ParticleGroup(data=tao.bunch_data("end"))
P0.drift_to_t()
P1.drift_to_t()
P1.plot("delta_z", "energy", bins=100)

In [ ]:
e1 = P1["mean_energy"]
e0 = P0["mean_energy"]
(e1 - e0)

In [ ]:
def gaussian(z, sigma=beam_init["sig_z"], mean=0):
    return 1 / np.sqrt(2 * np.pi) / sigma * np.exp(-0.5 * ((z - mean) / sigma) ** 2)


# 1. Choose a sampling grid
z_min, z_max, dz = -200e-6, 200e-6, 0.1e-6
z = np.arange(z_min, z_max + dz, dz)

# 2. Define / sample your functions
density = gaussian(z, mean=-0)  # Density

In [ ]:
iwake = wake.convolve(density, dz=dz)

In [ ]:
z_particles = P1["delta_z"]
de_particles = P1.energy - P0["mean_energy"]

zscale = 1 / 1e-6
fig, ax = plt.subplots()
ax.scatter(
    z_particles * zscale, de_particles, label="tracking", marker=".", color="black"
)

# ax.plot(z_conv * zscale, conv * N * e, color='red', label='convolution')

ax.plot(
    z * zscale, -iwake * beam_init["bunch_charge"], color="red", label="convolution"
)

ax.legend()
ax.set_xlim(z_particles.min() * zscale, z_particles.max() * zscale)
ax.set_xlabel(r"$z$ (µm)")
ax.set_ylabel(r"$W_z$ (eV/m)")

# Detailed wake


In [ ]:
iz = np.argsort(P0.z)

z = P0.z[iz]
weight = P0.weight[iz]
z[0], z[-1]

In [ ]:
%%time
zwake = apply_sr_wake(z, weight, wake.pseudomode)
len(zwake)

In [ ]:
fig, ax = plt.subplots()
ax.scatter(z * zscale, zwake, label="Python", color="black")
ax.scatter(
    z_particles * zscale, de_particles, label="Bmad tracking", marker=".", color="blue"
)
ax.legend()
ax.set_xlabel(r"$z$ (µm)")
ax.set_ylabel(r"$W_z$ (eV/m)")